# 03B — Búsqueda de hiperparámetros con Optuna (Continual Learning)

Este cuaderno automatiza la **optimización de hiperparámetros (HPO)** para métodos de aprendizaje continuo:
- **naive** (baseline, sin HPs),
- **ewc** (Elastic Weight Consolidation),
- **rehearsal** (rejuego con buffer),
- **rehearsal+ewc** (combinación).

Se apoya en:
- `configs/presets.yaml` (misma configuración que el resto de notebooks),
- `build_make_loader_fn` (carga CSV+runtime o H5 offline),
- `run_continual` (entrena & evalúa y guarda métricas).

---

### Métrica objetivo (minimizar)

$$
\textbf{Objetivo}
= \mathrm{MAE}_{\text{tarea final}}
+ \alpha \cdot \max\!\bigl(0,\, \text{OlvidoRelativo}\,\%\bigr)
$$


- **MAE_tarea final**: error en la **última** tarea (queremos aprender bien lo nuevo).
- **OlvidoRelativo %**: cuánto **empeora** la primera tarea tras aprender la segunda.
- **α**: peso del olvido (por defecto 0.5). Sube α si quieres penalizar más el olvido.

## ✅ Prerrequisitos
- `pip install optuna`
- Datos preparados (`tasks.json` o `tasks_balanced.json`).
- Idealmente H5 offline si `use_offline_spikes: true` en el preset.

<a id="toc"></a>

## 🧭 Índice
- [1) Imports y setup](#sec-01)
- [2) Carga de preset y construcción de modelo/transform](#sec-02)
- [3) Tareas y factory de loaders](#sec-03)
- [4) Métricas y objetivo para Optuna](#sec-04)
- [5) Espacios de búsqueda (por método)](#sec-05)
- [6) Estudio Optuna — un método concreto](#sec-06)
- [7) Estudio Optuna conjunto (elige método + HPs)](#sec-07)
- [8) Re-entrena con los mejores hiperparámetros](#sec-08)
- [9) Resumen rápido de runs (tabla)](#sec-09)


---

> **Consejo**: empieza con el preset `fast` y `N_TRIALS` pequeño; si todo va bien, sube `N_TRIALS` y/o las `epochs`.


<a id="sec-01"></a>

## 1) Imports y setup de entorno

- Limitamos hilos BLAS (`OMP`, `MKL`, `OPENBLAS`) para evitar sobrecarga de CPU.
- Configuramos **CUDA/TF32** para acelerar en GPUs NVIDIA.
- Insertamos la **raíz del repo** en `sys.path` para importar módulos locales.
- Comprobamos el **dispositivo** (`cuda`/`cpu`).

> Si notas que el equipo va justo de CPU, baja `torch.set_num_threads(4)` a `2`.

[↑ Volver al índice](#toc)

In [1]:
# Limitar threads BLAS (opcional)
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

from pathlib import Path
import sys, json, copy, time
import torch
import optuna

# Raíz del repo y sys.path
ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

# Proyecto
from src.utils import load_preset, build_make_loader_fn
from src.datasets import ImageTransform, AugmentConfig
from src.models import build_model
from src.runner import run_continual

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_num_threads(4)
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.set_float32_matmul_precision("high")

print("Device:", device)

/home/cesar/proyectos/TFM_SNN/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cuda


<a id="sec-02"></a>

## 2) Carga de preset y construcción de modelo/transform

- `PRESET` se carga desde `configs/presets.yaml`.
- Extraemos:
  - **Modelo** y el **transform** de imagen (`ImageTransform`) acorde a `img_w`, `img_h`, `to_gray`.
  - Parámetros de **codificación temporal** (`encoder`, `T`, `gain`) y **semilla**.
  - Flags de **carga de datos**: `use_offline_spikes` (H5), `encode_runtime` (codifica en GPU), `use_offline_balanced`.
  - Parámetros del **DataLoader** (workers, prefetch, pin_memory, etc.) y **balanceo online**.

- `make_model_fn(tfm)` devuelve el modelo (por ejemplo, `pilotnet_snn` con `beta/threshold`).

> Mantener aquí la **fuente de la verdad** del experimento (preset) ahorra inconsistencias respecto a otros notebooks.

[↑ Volver al índice](#toc)

In [2]:
PRESET = "fast"  # fast | std | accurate
CFG = load_preset(ROOT / "configs" / "presets.yaml", PRESET)

# Modelo / tfm
MODEL_NAME = CFG["model"]["name"]
tfm = ImageTransform(
    CFG["model"]["img_w"],
    CFG["model"]["img_h"],
    to_gray=bool(CFG["model"]["to_gray"]),
    crop_top=None
)

# Datos / codificación temporal
ENCODER = CFG["data"]["encoder"]
T       = int(CFG["data"]["T"])
GAIN    = float(CFG["data"]["gain"])
SEED    = int(CFG["data"]["seed"])

# Flags & loader
USE_OFFLINE_SPIKES = bool(CFG["data"].get("use_offline_spikes", False))
RUNTIME_ENCODE     = bool(CFG["data"].get("encode_runtime", False))

NUM_WORKERS = int(CFG["data"].get("num_workers") or 0)      # robusto
PREFETCH    = int(CFG["data"].get("prefetch_factor") or 2)  # <- casteo robusto
PIN_MEMORY  = bool(CFG["data"].get("pin_memory", True))
PERSISTENT  = bool(CFG["data"].get("persistent_workers", True))

AUG_CFG = AugmentConfig(**(CFG["data"].get("aug_train") or {})) \
          if CFG["data"].get("aug_train") else None

USE_ONLINE_BAL = bool(CFG["data"].get("balance_online", False))
BAL_BINS = int(CFG["data"].get("balance_bins") or 50)
BAL_EPS  = float(CFG["data"].get("balance_smooth_eps") or 1e-3)

print(f"[PRESET={PRESET}] model={MODEL_NAME} {tfm.w}x{tfm.h} gray={tfm.to_gray}")
print(f"[DATA] encoder={ENCODER} T={T} gain={GAIN} seed={SEED}")
print(f"[LOADER] workers={NUM_WORKERS} prefetch={PREFETCH} pin={PIN_MEMORY} persistent={PERSISTENT}")
print(f"[BALANCE] online={USE_ONLINE_BAL} bins={BAL_BINS}")
print(f"[RUNTIME_ENCODE] {RUNTIME_ENCODE} | [OFFLINE_SPIKES] {USE_OFFLINE_SPIKES}")

def make_model_fn(tfm):
    # kwargs específicos de pilotnet_snn; ignorados para otros
    return build_model(MODEL_NAME, tfm, beta=0.9, threshold=0.5)


[PRESET=fast] model=pilotnet_snn 200x66 gray=True
[DATA] encoder=rate T=10 gain=0.5 seed=42
[LOADER] workers=8 prefetch=2 pin=True persistent=True
[BALANCE] online=False bins=50
[RUNTIME_ENCODE] False | [OFFLINE_SPIKES] True


<a id="sec-03"></a>

## 3) Tareas y factory de loaders

- Leemos `tasks.json` o `tasks_balanced.json` desde `data/processed/`.
- Cada tarea apunta a CSV/H5 de `train/val/test`.
- Si `use_offline_spikes: true`, `build_make_loader_fn` **elige H5**; si no, usa **CSV + runtime encode** en GPU.

El **wrapper** `make_loader_fn(...)` simplemente pasa argumentos al factory real; así el `runner` puede inyectar kwargs (augment, balanceo online, etc.) sin reescribir nada aquí.

> Si activas **offline balanceado**, el **train** debería ser `train_balanced.csv` o el H5 derivado. El notebook advierte si no coincide.

[↑ Volver al índice](#toc)

In [3]:
# Leer tasks.json / tasks_balanced.json (elige según el preset)
from pathlib import Path as _P
import json

PROC = ROOT / "data" / "processed"

USE_BALANCED = bool(CFG.get("prep_offline", {}).get("use_balanced_tasks", False))
tb_name = (CFG.get("prep", {}).get("tasks_balanced_file_name") or "tasks_balanced.json")
t_name  = (CFG.get("prep", {}).get("tasks_file_name")           or "tasks.json")

cand_bal = PROC / tb_name
cand_std = PROC / t_name
TASKS_FILE = cand_bal if (USE_BALANCED and cand_bal.exists()) else cand_std

with open(TASKS_FILE, "r", encoding="utf-8") as f:
    tasks_json = json.load(f)
task_list = [{"name": n, "paths": tasks_json["splits"][n]} for n in tasks_json["tasks_order"]]

print("Usando:", TASKS_FILE.name)
print("Tareas y TRAIN CSV/H5 a usar:")
for t in task_list:
    print(f" - {t['name']}: {_P(t['paths']['train']).name}")

# Guardarraíl: si balanced, exigir train_balanced.csv
if USE_BALANCED:
    for t in task_list:
        train_path = _P(tasks_json["splits"][t["name"]]["train"])
        if train_path.name != "train_balanced.csv":
            raise RuntimeError(
                f"[{t['name']}] Esperaba 'train_balanced.csv' en modo balanced, pero encontré '{train_path.name}'."
            )

# Si usas H5 offline, chequear que existan
if USE_OFFLINE_SPIKES:
    mw, mh = CFG["model"]["img_w"], CFG["model"]["img_h"]
    color = "gray" if CFG["model"]["to_gray"] else "rgb"
    gain_tag = (GAIN if ENCODER == "rate" else 0)
    missing = []
    for t in task_list:
        base = PROC / t["name"]
        for split in ("train", "val", "test"):
            p = base / f"{split}_{ENCODER}_T{T}_gain{gain_tag}_{color}_{mw}x{mh}.h5"
            if not p.exists():
                missing.append(str(p))
    if missing:
        print("[WARN] Faltan H5. Genera primero con 02_ENCODE_OFFLINE.ipynb (o tools/encode_tasks.py).")

# Factory de loaders con kwargs del preset
from src.utils import build_make_loader_fn

_raw_make_loader_fn = build_make_loader_fn(
    root=ROOT, use_offline_spikes=USE_OFFLINE_SPIKES, encode_runtime=RUNTIME_ENCODE,
)

_DL_KW = dict(
    num_workers=NUM_WORKERS,
    prefetch_factor=PREFETCH,
    pin_memory=PIN_MEMORY,
    persistent_workers=PERSISTENT,
    aug_train=AUG_CFG,
    balance_train=USE_ONLINE_BAL,
    balance_bins=BAL_BINS,
    balance_smooth_eps=BAL_EPS,
)

def make_loader_fn(task, batch_size, encoder, T, gain, tfm, seed, **dl_kwargs):
    """Wrapper pass-through: el runner añade dl_kwargs; aquí solo los propagamos."""
    return _raw_make_loader_fn(
        task=task, batch_size=batch_size, encoder=encoder, T=T, gain=gain, tfm=tfm, seed=seed,
        **{**_DL_KW, **dl_kwargs}
    )

print("make_loader_fn listo.")


Usando: tasks.json
Tareas y TRAIN CSV/H5 a usar:
 - circuito1: train.csv
 - circuito2: train.csv
make_loader_fn listo.


<a id="sec-04"></a>

## 4) Métricas y objetivo para Optuna

**Cómo se calcula:**
1. Cargamos `continual_results.json` del run.
2. Detectamos **primera** y **última** tarea (heurística simple: la última no tiene claves `after_*`).
3. Extraemos:
   - `c1_mae`: MAE en la **primera** tarea en su propio test.
   - `c1_after_c2_mae`: MAE de la **primera** *después* de aprender la segunda (olvido).
   - `c2_mae`: MAE de la **última** tarea.
4. **Olvido relativo %** = \((c1\_after\_c2 - c1\_mae) / c1\_mae \times 100\).

**Objetivo** = `c2_mae + ALPHA_FORGET * max(0, olvido_relativo_%)`  
- **Minimizar** este valor favorece: buen rendimiento en la **tarea final** y **poco olvido** de la primera.
- Ajusta `ALPHA_FORGET` si quieres **penalizar más** el olvido (sube α) o **priorizar** la tarea nueva (baja α).

> Si el JSON no existe o faltan métricas, devolvemos `inf` para que ese trial no gane.

[↑ Volver al índice](#toc)

In [4]:
# === Métricas y objetivo: usa utilidades del repo ===
from src.utils_exp import extract_metrics, safe_read_json
from src.hpo import objective_value, composite_objective
from pathlib import Path

ALPHA_FORGET = 0.5  # peso del olvido relativo en el objetivo compuesto

def _load_results(out_dir: Path) -> dict:
    """Lee outputs/<exp>/continual_results.json con manejo robusto."""
    return safe_read_json(Path(out_dir) / "continual_results.json")


<a id="sec-05"></a>

## 5) Espacios de búsqueda (por método)

Definimos qué hiperparámetros explora **Optuna** en cada método:

- **ewc**:
  - `lam` (*lambda*): [3e8, 2e9] (log-uniform). Penalización de estabilidad (evita olvidar).
  - `fisher_batches`: [200, 1200]. Cuánta info de Fisher acumulamos (coste ↑).

- **rehearsal**:
  - `buffer_size`: [1000, 8000]. Tamaño de memoria de rejuego.
  - `replay_ratio`: [0.05, 0.4]. Fracción de muestras de memoria por minibatch.

- **rehearsal+ewc**: combina ambos sets.

- **naive**: sin hiperparámetros (sirve como línea base).

> Rango amplio = más tiempo pero más opciones. Ajusta rangos cuando tengas intuición.

[↑ Volver al índice](#toc)

In [5]:
def suggest_params_for_method(trial: optuna.Trial, method: str) -> dict:
    method = method.lower()
    if method == "ewc":
        lam = trial.suggest_float("lam", 3e8, 2e9, log=True)
        fisher_batches = trial.suggest_int("fisher_batches", 200, 1200, step=100)
        return {"lam": lam, "fisher_batches": fisher_batches}
    elif method == "rehearsal":
        buffer_size  = trial.suggest_int("buffer_size", 1000, 8000, step=1000)
        replay_ratio = trial.suggest_float("replay_ratio", 0.05, 0.4, step=0.05)
        return {"buffer_size": buffer_size, "replay_ratio": replay_ratio}
    elif method == "rehearsal+ewc":
        buffer_size  = trial.suggest_int("buffer_size", 1000, 8000, step=1000)
        replay_ratio = trial.suggest_float("replay_ratio", 0.05, 0.4, step=0.05)
        lam = trial.suggest_float("lam", 3e8, 2e9, log=True)
        fisher_batches = trial.suggest_int("fisher_batches", 200, 1200, step=100)
        return {"buffer_size": buffer_size, "replay_ratio": replay_ratio, "lam": lam, "fisher_batches": fisher_batches}
    else:  # naive (sin HPs)
        return {}

<a id="sec-06"></a>

## 6) Estudio Optuna — un método concreto

- **Study**: contenedor de la búsqueda.
- **Trial**: una configuración (punto) en el espacio de HPs.
- **Objective**: función que entrena/evalúa y devuelve un **valor a minimizar**.

Parámetros clave:
- `METHOD_TO_OPTIMIZE`: `"ewc"`, `"rehearsal"`, `"rehearsal+ewc"` o `"naive"`.
- `N_TRIALS`: nº de configuraciones a probar.
- `HPO_EPOCHS`: si no es `None`, **sobrescribe** las `epochs` del preset **solo durante HPO** (acelera la búsqueda). Luego reentrenas a tope en la Sección 8.

**Qué hace cada trial:**
1. Sugerir HPs (`suggest_*`).
2. Construir `cfg` con esos HPs (y `epochs` reducidas si `HPO_EPOCHS`).
3. Ejecutar `run_continual(...)`.
4. Leer `continual_results.json`, calcular métrica objetivo y devolvérsela a Optuna.

**Salida:**
- `study.best_params`: mejores HPs.
- `study.best_value`: valor objetivo mínimo.
- `study.best_trial.user_attrs`: metadatos (ruta del experimento, métricas) que guardamos nosotros.

> **Tip:** Para runs largos añade *pruning* o una base de datos (`optuna.create_study(storage=...)`) si quieres **reanudar** búsquedas en varias sesiones.

[↑ Volver al índice](#toc)

In [6]:
# Configuración del estudio
METHOD_TO_OPTIMIZE = "ewc"   # "naive" | "ewc" | "rehearsal" | "rehearsal+ewc"
N_TRIALS = 8                 # súbelo cuando estés satisfecho con tiempos/estabilidad
HPO_EPOCHS = None            # None -> usar epochs del preset; o pon un int (ej. 3) para acelerar
REHEARSAL_NAMES = ("rehearsal", "rehearsal+ewc")

def build_cfg_with_method(base_cfg: dict, method_name: str, params: dict, hpo_epochs: int|None):
    cfg = copy.deepcopy(base_cfg)
    cfg["continual"]["method"] = method_name
    cfg["continual"]["params"] = params or {}

    if hpo_epochs is not None:
        cfg["optim"]["epochs"] = int(hpo_epochs)
    return cfg

def run_one_cfg(cfg: dict) -> tuple[Path, dict, dict]:
    out_dir, res = run_continual(
        task_list=task_list,
        make_loader_fn=make_loader_fn,
        make_model_fn=make_model_fn,
        tfm=tfm,
        cfg=cfg,
        preset_name=PRESET,
        out_root=ROOT / "outputs",
        verbose=True,
    )
    results = _load_results(out_dir) or (res if isinstance(res, dict) else {})
    return out_dir, res, results

def optuna_objective(trial: optuna.Trial):
    params = suggest_params_for_method(trial, METHOD_TO_OPTIMIZE)
    cfg_i  = build_cfg_with_method(CFG, METHOD_TO_OPTIMIZE, params, HPO_EPOCHS)

    # === Hotfix selectivo para estabilidad con replay ===
    if METHOD_TO_OPTIMIZE.lower() in REHEARSAL_NAMES:
        cfg_i["data"]["persistent_workers"] = False
        # Opcionales si vieras inestabilidad puntual:
        # cfg_i["optim"]["amp"] = False
        # cfg_i["data"]["pin_memory"] = False
        # cfg_i["data"]["num_workers"] = min(int(cfg_i["data"].get("num_workers") or 0), 2)

    cfg_i.setdefault("naming", {})
    tag = f"{METHOD_TO_OPTIMIZE}_hpo_t{trial.number}"
    if METHOD_TO_OPTIMIZE in ("ewc", "rehearsal+ewc") and "lam" in params:
        tag += f"_lam_{params['lam']:.1e}"
    cfg_i["naming"]["tag"] = tag

    try:
        out_dir, _, results = run_one_cfg(cfg_i)
        metrics = extract_metrics(results)
        val = composite_objective(metrics, ALPHA_FORGET)
        trial.set_user_attr("out_dir", str(out_dir))
        trial.set_user_attr("metrics", metrics)
        trial.set_user_attr("method", METHOD_TO_OPTIMIZE)
        trial.set_user_attr("params", params)
        return val
    except Exception as e:
        # No tires abajo todo el estudio por un trial problemático
        trial.set_user_attr("error", repr(e))
        return float("inf")
    finally:
        # Limpieza ligera entre trials (evita fragmentación de memoria GPU)
        import gc, time
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        time.sleep(0.5)



# --- Persistencia Optuna en SQLite ---
OPTUNA_DIR = ROOT / "outputs" / "optuna"
OPTUNA_DIR.mkdir(parents=True, exist_ok=True)

DB_PATH = OPTUNA_DIR / f"hpo_{METHOD_TO_OPTIMIZE}_{PRESET}_{MODEL_NAME}_{ENCODER}_T{T}_g{GAIN}.sqlite"
STORAGE = f"sqlite:///{DB_PATH}"

study = optuna.create_study(
    direction="minimize",
    study_name=f"HPO_{METHOD_TO_OPTIMIZE}_{PRESET}_{ENCODER}_T{T}",
    storage=STORAGE,
    load_if_exists=True,
)

# Opcional: sampler/pruner (puedes añadirlos si quieres)
# from optuna.samplers import TPESampler
# from optuna.pruners import MedianPruner
# study.sampler = TPESampler(seed=SEED)
# study.pruner  = MedianPruner(n_startup_trials=3)

study.optimize(optuna_objective, n_trials=N_TRIALS, show_progress_bar=True)

print("SQLite:", DB_PATH)
print("Best value:", study.best_value)
print("Best params:", study.best_params)
print("Best attrs:", study.best_trial.user_attrs)

# Guardar trazabilidad de intentos a CSV
df_trials = study.trials_dataframe(attrs=("number","value","state","params","user_attrs"))
df_trials.to_csv(OPTUNA_DIR / f"{DB_PATH.stem}_trials.csv", index=False)


print("Best value:", study.best_value)
print("Best params:", study.best_params)
print("Best attrs:", study.best_trial.user_attrs)

[I 2025-09-28 16:33:05,145] A new study created in RDB with name: HPO_ewc_fast_rate_T10
  0%|          | 0/8 [00:00<?, ?it/s]


--- Tarea 1/2: circuito1 | preset=fast | method=ewc_lam_5e+08 | B=64 T=10 AMP=True | enc=rate ---



--- Tarea 2/2: circuito2 | preset=fast | method=ewc_lam_5e+08 | B=64 T=10 AMP=True | enc=rate ---


Best trial: 0. Best value: 0.22288:  12%|█▎        | 1/8 [02:36<18:17, 156.84s/it]

[I 2025-09-28 16:35:41,983] Trial 0 finished with value: 0.22288037992292836 and parameters: {'lam': 479389757.05559325, 'fisher_batches': 1100}. Best is trial 0 with value: 0.22288037992292836.

--- Tarea 1/2: circuito1 | preset=fast | method=ewc_lam_7e+08 | B=64 T=10 AMP=True | enc=rate ---


...

Best trial: 0. Best value: 0.22288:  88%|████████▊ | 7/8 [20:43<02:55, 175.57s/it]

[I 2025-09-28 16:53:48,286] Trial 6 finished with value: 0.4713248420418903 and parameters: {'lam': 360004291.6972955, 'fisher_batches': 800}. Best is trial 0 with value: 0.22288037992292836.

--- Tarea 1/2: circuito1 | preset=fast | method=ewc_lam_3e+08 | B=64 T=10 AMP=True | enc=rate ---



--- Tarea 2/2: circuito2 | preset=fast | method=ewc_lam_3e+08 | B=64 T=10 AMP=True | enc=rate ---


Best trial: 0. Best value: 0.22288: 100%|██████████| 8/8 [23:23<00:00, 175.45s/it]

[I 2025-09-28 16:56:28,712] Trial 7 finished with value: 0.41779417044634304 and parameters: {'lam': 330897635.9713998, 'fisher_batches': 400}. Best is trial 0 with value: 0.22288037992292836.
SQLite: /home/cesar/proyectos/TFM_SNN/outputs/optuna/hpo_ewc_fast_pilotnet_snn_rate_T10_g0.5.sqlite
Best value: 0.22288037992292836
Best params: {'lam': 479389757.05559325, 'fisher_batches': 1100}
Best attrs: {'method': 'ewc', 'metrics': {'c1': 'circuito1', 'c2': 'circuito2', 'c1_mae': 0.1717126432347124, 'c1_after_c2_mae': 0.1710578842833708, 'forget_rel_%': -0.3813108569102919, 'c2_mae': 0.22288037992292836}, 'out_dir': '/home/cesar/proyectos/TFM_SNN/outputs/continual_fast_ewc_lam_5e+08_lam_5e+08_ewc_hpo_t0_lam_4.8e+08_rate_model-PilotNetSNN_66x200_gray_seed_42', 'params': {'lam': 479389757.05559325, 'fisher_batches': 1100}}
Best value: 0.22288037992292836
Best params: {'lam': 479389757.05559325, 'fisher_batches': 1100}
Best attrs: {'method': 'ewc', 'metrics': {'c1': 'circuito1', 'c2': 'circuit

<a id="sec-07"></a>

## 7) Estudio Optuna conjunto (elige método + HPs)

Si `RUN_JOINT=True`, el **trial** también elige el **método**:
- `method ∈ {naive, ewc, rehearsal, rehearsal+ewc}`
- Y luego sus HPs correspondientes.

Útil cuando:
- No sabes qué método es mejor en tu dataset.
- Quieres una **comparativa automática** con el mismo presupuesto de cómputo.

> Empieza con `RUN_JOINT=False` para validar el flujo con un único método. Luego enciéndelo y sube `N_TRIALS_JOINT`.

[↑ Volver al índice](#toc)

In [7]:
RUN_JOINT = False   # Pon True si quieres lanzar el estudio conjunto
N_TRIALS_JOINT = 10

def optuna_objective_joint(trial: optuna.Trial):
    method = trial.suggest_categorical("method", ["naive","ewc","rehearsal","rehearsal+ewc"])
    params = suggest_params_for_method(trial, method)
    cfg_i  = build_cfg_with_method(CFG, method, params, HPO_EPOCHS)

    # === Hotfix selectivo (solo métodos con replay) ===
    if method.lower() in REHEARSAL_NAMES:
        cfg_i["data"]["persistent_workers"] = False
        # Opcionales si hiciera falta:
        # cfg_i["optim"]["amp"] = False
        # cfg_i["data"]["pin_memory"] = False
        # cfg_i["data"]["num_workers"] = min(int(cfg_i["data"].get("num_workers") or 0), 2)

    cfg_i.setdefault("naming", {})
    tag = f"{method}_hpo_t{trial.number}"
    if method in ("ewc", "rehearsal+ewc") and "lam" in params:
        tag += f"_lam_{params['lam']:.1e}"
    cfg_i["naming"]["tag"] = tag

    try:
        out_dir, _, results = run_one_cfg(cfg_i)
        metrics = extract_metrics(results)
        val = objective_value(metrics, ALPHA_FORGET)  # tu versión conjunta
        trial.set_user_attr("out_dir", str(out_dir))
        trial.set_user_attr("metrics", metrics)
        trial.set_user_attr("method", method)
        trial.set_user_attr("params", params)
        return val
    except Exception as e:
        trial.set_user_attr("error", repr(e))
        return float("inf")
    finally:
        import gc, time
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        time.sleep(0.5)



if RUN_JOINT:
    study_joint = optuna.create_study(direction="minimize", study_name="HPO_joint")
    study_joint.optimize(optuna_objective_joint, n_trials=N_TRIALS_JOINT, show_progress_bar=True)
    print("Best value:", study_joint.best_value)
    print("Best params:", study_joint.best_params)
    print("Best attrs:", study_joint.best_trial.user_attrs)
else:
    print("RUN_JOINT=False — omitido.")

RUN_JOINT=False — omitido.


<a id="sec-08"></a>

## 8) Re-entrena con los mejores hiperparámetros

- Coge `study.best_params` y `METHOD_TO_OPTIMIZE`.
- Reconstruye `cfg_best` con el **método ganador + HPs**.
- (Opcional) Restablece `epochs` del preset si usaste `HPO_EPOCHS` para acelerar.
- Lanza `run_continual(...)` **a pleno rendimiento**.
- Muestra métricas finales y la **carpeta de salida**.

> Así separas la **búsqueda rápida** (pocas epochs) del **entrenamiento serio** (epochs del preset).

[↑ Volver al índice](#toc)

In [8]:
# Usa el mejor del estudio por método (arriba)
BEST_PARAMS = study.best_params
BEST_METHOD = METHOD_TO_OPTIMIZE

print("Mejor método:", BEST_METHOD)
print("Mejores HPs:", BEST_PARAMS)

cfg_best = copy.deepcopy(CFG)
cfg_best["continual"]["method"] = BEST_METHOD
cfg_best["continual"]["params"] = BEST_PARAMS

# (Opcional) restablecer epochs al valor del preset si redujiste para HPO
# cfg_best["optim"]["epochs"] = load_preset(ROOT / "configs" / "presets.yaml", PRESET)["optim"]["epochs"]

out_dir, _, results = run_one_cfg(cfg_best)
metrics = extract_metrics(results)
print("Resultados finales (re-train):", metrics)
print("Guardado en:", out_dir)

Mejor método: ewc
Mejores HPs: {'lam': 479389757.05559325, 'fisher_batches': 1100}

--- Tarea 1/2: circuito1 | preset=fast | method=ewc_lam_5e+08 | B=64 T=10 AMP=True | enc=rate ---



--- Tarea 2/2: circuito2 | preset=fast | method=ewc_lam_5e+08 | B=64 T=10 AMP=True | enc=rate ---


Resultados finales (re-train): {'c1': 'circuito1', 'c2': 'circuito2', 'c1_mae': 0.17128969528016497, 'c1_after_c2_mae': 0.1716454872878574, 'forget_rel_%': 0.20771360887208243, 'c2_mae': 0.22322455726644044}
Guardado en: /home/cesar/proyectos/TFM_SNN/outputs/continual_fast_ewc_lam_5e+08_lam_5e+08_rate_model-PilotNetSNN_66x200_gray_seed_42


<a id="sec-09"></a>

## 9) Resumen rápido de runs (tabla)

- Recorremos `outputs/continual_*` y leemos `continual_results.json`.
- Extraemos y tabulamos:
  - `preset`, `method`, `encoder`, `seed`, (y `lambda` si aplica),
  - `c1_mae`, `c1_after_c2_mae`, `forget_rel_%`, `c2_mae`.

**Cómo leerla:**
- **`c2_mae`** bajo → aprende bien la última tarea.
- **`forget_rel_%`** bajo → **poco olvido** de la primera.
- Filtra por preset/método para comparar **manzanas con manzanas**.

> Consejo: Exporta a CSV/Parquet si quieres hacer gráficas comparativas a posteriori.

[↑ Volver al índice](#toc)

In [9]:
# === Tabla mínima para HPO / inspección rápida ===
from src.utils_exp import build_runs_df
import pandas as pd

outputs_root = ROOT / "outputs"
df = build_runs_df(outputs_root)

print(f"runs en resumen: {len(df)}")
if df.empty:
    print("No hay filas (¿no existen JSONs o solo hubo 1 tarea por run?).")
else:
    # columnas clave para tuning; mantiene el nombre 'forget_rel_%' por compatibilidad
    view = df.rename(columns={"c1_forgetting_mae_rel_%": "forget_rel_%"}).loc[:, [
        "exp","preset","method","lambda","encoder","model","seed",
        "c1_mae","c1_after_c2_mae","forget_rel_%","c2_mae"
    ]]
    display(view.sort_values(["preset","method","encoder","lambda"], na_position="last", ignore_index=True))


runs en resumen: 16


,exp,preset,method,lambda,encoder,model,seed,c1_mae,c1_after_c2_mae,forget_rel_%,c2_mae
0,continual_fast_ewc_lam_1e+09_lam_1e+09_rate_mo...,fast,ewc,1e+09,rate,PilotNetSNN_66x200_gray,42,0.119421,0.126423,5.863543,0.170004
1,continual_fast_ewc_lam_1e+09_lam_1e+09_rate_mo...,fast,ewc,1e+09,rate,PilotNetSNN_66x200_gray,43,0.096688,0.115334,19.285470,0.152892
2,continual_fast_ewc_lam_5e+08_lam_5e+08_rate_mo...,fast,ewc,5e+08,rate,PilotNetSNN_66x200_gray,42,0.171290,0.171645,0.207714,0.223225
3,continual_fast_ewc_lam_7e+08_lam_7e+08_rate_mo...,fast,ewc,7e+08,rate,PilotNetSNN_66x200_gray,42,0.119421,0.145897,22.170299,0.173348
4,continual_fast_ewc_lam_5e+08_lam_5e+08_ewc_hpo...,fast,ewc_ewc_hpo_t0_lam_4.8e+08,5e+08,rate,PilotNetSNN_66x200_gray,42,0.171713,0.171058,-0.381311,0.222880
5,continual_fast_ewc_lam_7e+08_lam_7e+08_ewc_hpo...,fast,ewc_ewc_hpo_t1_lam_7.1e+08,7e+08,rate,PilotNetSNN_66x200_gray,42,0.171290,0.171485,0.114275,0.223129
6,continual_fast_ewc_lam_4e+08_lam_4e+08_ewc_hpo...,fast,ewc_ewc_hpo_t2_lam_4.5e+08,4e+08,rate,PilotNetSNN_66x200_gray,42,0.171290,0.171640,0.204753,0.223222
7,continual_fast_ewc_lam_6e+08_lam_6e+08_ewc_hpo...,fast,ewc_ewc_hpo_t3_lam_6.4e+08,6e+08,rate,PilotNetSNN_66x200_gray,42,0.171290,0.171794,0.294161,0.223314
8,continual_fast_ewc_lam_2e+09_lam_2e+09_ewc_hpo...,fast,ewc_ewc_hpo_t4_lam_1.8e+09,2e+09,rate,PilotNetSNN_66x200_gray,42,0.171290,0.171313,0.013333,0.223025
9,continual_fast_ewc_lam_8e+08_lam_8e+08_ewc_hpo...,fast,ewc_ewc_hpo_t5_lam_8.1e+08,8e+08,rate,PilotNetSNN_66x200_gray,42,0.171290,0.171527,0.138447,0.223153


## Apéndice — Optuna en 90 segundos

- **Study**: el proyecto de HPO (contiene todos los trials).
- **Trial**: una evaluación con un conjunto de HPs (`suggest_int`, `suggest_float`, etc.).
- **Sampler**: estrategia para elegir el siguiente punto (por defecto, TPE).
- **Pruner**: **corta** trials que pintan mal (acelera búsquedas largas).
- **Storage**: base de datos (SQLite, PostgreSQL) para **reanudar** y/o **paralelizar**.

### Reanudar búsquedas
Puedes crear el estudio con almacenamiento:
```python
study = optuna.create_study(
    direction="minimize",
    study_name="HPO_ewc",
    storage=f"sqlite:///{ROOT/'outputs'/'optuna_ewc.sqlite'}",
    load_if_exists=True,
)


In [10]:
from src.hpo import objective_value, composite_objective
m = {"c2_mae": 0.16, "forget_rel_%": 12.5}
print(objective_value(m, key="forget_rel_%"))         # 12.5
print(objective_value(m, key="c1_forgetting_mae_rel_%"))  # 12.5 (alias OK)
print(composite_objective(m, alpha=0.5))              # 0.16 + 0.5*12.5 = 6.41


12.5
12.5
6.41
